# Part I

Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

SELECT name 
FROM `Facilities` 
WHERE membercost > 0

Q2: How many facilities do not charge a fee to members?

In [ ]:
SELECT COUNT(name) 
FROM `Facilities` 
WHERE membercost = 0

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [ ]:
SELECT *
FROM `Facilities` 
WHERE membercost > 0
AND membercost < 0.2 * monthlymaintenance

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. 

In [ ]:
SELECT *
FROM `Facilities` 
WHERE facid IN (1,5)

Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [ ]:
SELECT name,
CASE WHEN monthlymaintenance > 100 THEN "expensive"
ELSE "cheap"
END AS cost_label
FROM country_club.Facilities 

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. 

In [ ]:
SELECT firstname, surname
FROM country_club.Members
WHERE joindate = (SELECT MAX(joindate) FROM country_club.Members)

Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [ ]:
SELECT DISTINCT f.name, m.firstname, m.surname
FROM country_club.Bookings b
LEFT JOIN country_club.Facilities f
ON b.facid = f.facid
LEFT JOIN country_club.Members m
ON b.memid = m.memid
WHERE f.name LIKE "%Tennis%"
ORDER BY m.firstname ASC

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [ ]:
SELECT b.bookid, f.name as facility_name, m.firstname, m.surname, 
CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
ELSE f.membercost * b.slots
END AS booking_cost
FROM country_club.Bookings b
LEFT JOIN country_club.Facilities f
ON b.facid = f.facid
LEFT JOIN country_club.Members m
ON b.memid = m.memid
WHERE ((b.memid = 0 AND f.guestcost * b.slots > 30) OR (b.memid > 0 AND f.membercost * b.slots > 30))
AND starttime LIKE "2012-09-14%"
ORDER BY booking_cost DESC

Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
SELECT s.bookid, s.facility_name, s.firstname, s.surname, s.booking_cost
FROM(
    SELECT b.bookid, f.name as facility_name, m.firstname, m.surname, b.starttime,
    CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
    ELSE f.membercost * b.slots
    END AS booking_cost
    FROM country_club.Bookings b
    LEFT JOIN country_club.Facilities f
    ON b.facid = f.facid
    LEFT JOIN country_club.Members m
    ON b.memid = m.memid) s

WHERE s.booking_cost > 30
AND s.starttime LIKE "2012-09-14%"
ORDER BY s.booking_cost DESC

# Part II

In [22]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:////Users/nataliiashcheglova/Documents/Springboard/data-science/Country Club/sqlite_db_pythonsqlite.db')

In [23]:
# Checking if conection works
cc_data = pd.read_sql_query("SELECT * FROM Facilities LIMIT 5", engine)

In [24]:
cc_data

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [33]:
q10 = pd.read_sql_query("SELECT name as facility_name, sum(revenue) as total_revenue FROM(SELECT Bookings.bookid, Facilities.name, CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost * Bookings.slots ELSE Facilities.membercost * Bookings.slots END AS revenue FROM Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid GROUP BY Facilities.name, Bookings.bookid) GROUP BY name", engine)
q10[q10['total_revenue']<1000]

,facility_name,total_revenue
3,Pool Table,270.0
4,Snooker Table,240.0
6,Table Tennis,180.0


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [26]:
q11 = pd.read_sql_query("SELECT m.memid, m.firstname as member_name, m.surname as member_surname, CASE WHEN r.firstname is NULL THEN 'No recommender' ELSE r.firstname  end as recommended_by_name, CASE WHEN r.surname is NULL THEN 'No recommender' ELSE r.surname  end as recommended_by_surname FROM Members as m LEFT JOIN Members as r ON m.recommendedby = r.memid WHERE m.memid != 0 ORDER BY m.surname asc, m.firstname asc",engine)
q11

,memid,member_name,member_surname,recommended_by_name,recommended_by_surname
0,15,Florence,Bader,Ponder,Stibbons
1,12,Anne,Baker,Ponder,Stibbons
2,16,Timothy,Baker,Jemima,Farrell
3,8,Tim,Boothe,Tim,Rownam
4,5,Gerald,Butters,Darren,Smith
5,22,Joan,Coplin,Timothy,Baker
6,36,Erica,Crumpet,Tracy,Smith
7,7,Nancy,Dare,Janice,Joplette
8,28,David,Farrell,No recommender,No recommender
9,13,Jemima,Farrell,No recommender,No recommender


Q11: Find the facilities with their usage by member, but not guests

In [34]:
q12 = pd.read_sql_query("SELECT Facilities.name as facility_name, Members.firstname, Members.surname, sum(Bookings.slots) as slots_used FROM Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid LEFT JOIN Members ON Bookings.memid = Members.memid WHERE Bookings.memid > 0 GROUP BY Facilities.name, Members.firstname, Members.surname ORDER BY Members.surname, Members.firstname",engine)
q12

,facility_name,firstname,surname,slots_used
0,Badminton Court,Florence,Bader,27
1,Massage Room 2,Florence,Bader,4
2,Pool Table,Florence,Bader,23
3,Snooker Table,Florence,Bader,66
4,Squash Court,Florence,Bader,4
...,...,...,...,...
197,Squash Court,Hyacinth,Tupperware,2
198,Badminton Court,Henry,Worthington-Smyth,15
199,Massage Room 1,Henry,Worthington-Smyth,2
200,Pool Table,Henry,Worthington-Smyth,37


Q13: Find the facilities usage by month, but not guests 

In [40]:
q13 = pd.read_sql_query("SELECT strftime('%Y',Bookings.starttime) as month, Facilities.name as facility_name, sum(Bookings.slots)/2 as hours_booked FROM Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid WHERE Bookings.memid > 0 GROUP BY month, Facilities.name ORDER BY month desc",engine)
q13



,month,facility_name,hours_booked
0,2012,Badminton Court,543
1,2012,Massage Room 1,442
2,2012,Massage Room 2,27
3,2012,Pool Table,428
4,2012,Snooker Table,430
5,2012,Squash Court,209
6,2012,Table Tennis,397
7,2012,Tennis Court 1,478
8,2012,Tennis Court 2,441
